In [ ]:
import pickle
fpaths = [
    '/group/jug/ashesh/data/paper_stats/Test_P64_G32_M50_Sk8/calib_stats_training_disentangle_2404_D21-M3-S0-L8_6.pkl',
'/group/jug/ashesh/data/paper_stats/Test_P64_G32_M50_Sk8/calib_stats_training_disentangle_2404_D25-M3-S0-L8_143.pkl',
'/group/jug/ashesh/data/paper_stats/Test_P64_G32_M50_Sk8/calib_stats_training_disentangle_2404_D25-M3-S0-L8_111.pkl',
'/group/jug/ashesh/data/paper_stats/Test_P64_G32_M50_Sk8/calib_stats_training_disentangle_2404_D25-M3-S0-L8_97.pkl',
'/group/jug/ashesh/data/paper_stats/Test_P64_G32_M50_Sk8/calib_stats_training_disentangle_2404_D25-M3-S0-L8_120.pkl',
'/group/jug/ashesh/data/paper_stats/Test_P64_G32_M50_Sk8/calib_stats_training_disentangle_2405_D25-M3-S0-L8_2.pkl',
'/group/jug/ashesh/data/paper_stats/Test_P64_G32_M50_Sk8/calib_stats_training_disentangle_2404_D25-M3-S0-L8_125.pkl',
'/group/jug/ashesh/data/paper_stats/Test_P64_G32_M50_Sk8/calib_stats_training_disentangle_2405_D25-M3-S0-L8_3.pkl',
'/group/jug/ashesh/data/paper_stats/Test_P64_G32_M50_Sk8/calib_stats_training_disentangle_2404_D25-M3-S0-L8_139.pkl',

]
calib_fpath =fpaths[9]
with open(calib_fpath, 'rb') as f:
    calib_stats = pickle.load(f)


In [ ]:
from disentangle.analysis.paper_plots import get_first_index, get_last_index
q_s = 0.001
q_e = 0.999


In [ ]:
import matplotlib.pyplot as plt
import os

_,ax = plt.subplots(figsize=(2,2))
for ch_idx in range(len(calib_stats['calib_stats'])):
    tmp_stats = calib_stats['calib_stats'][ch_idx]
    tmp_rmv = tmp_stats['rmv']
    tmp_rmse = tmp_stats['rmse']
    count = tmp_stats['bin_count']
    first_idx = get_first_index(count, q_s)
    last_idx = get_last_index(count, q_e)
    plt.plot(tmp_rmv[first_idx:-last_idx],
            tmp_rmse[first_idx:-last_idx],
            '-+',
            label='C{}'.format(ch_idx)
            )
plt.tight_layout()
fname = os.path.basename(calib_fpath).replace('.pkl','.png')
fpath = os.path.join('/group/jug/ashesh/naturemethods/calibration', fname)
plt.savefig(fpath, dpi=200, bbox_inches='tight')
print('Saved to {}'.format(fpath))
plt.legend()


In [ ]:
import os
